In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
# name_sheet = pd.read_csv('data/name_sheet2.csv')
city_name = input("请输入城市名称 (例如: SYD): ").upper()
if city_name.upper() == "SYD":
    name_sheet_path = 'data/name_sheet_syd.csv'
elif city_name.upper() == "ADL":
    name_sheet_path = 'data/name_sheet_ade.csv'
elif city_name.upper() == "BNE":
    name_sheet_path = 'data/name_sheet_bne.csv'
else:
    print(f"未找到城市 {city_name} 对应的name_sheet文件，将使用默认文件")
    name_sheet_path = 'data/name_sheet2.csv'

In [50]:
name_sheet = pd.read_csv(name_sheet_path)

In [4]:
file_path = r"output\BPE.xlsx"

# # 从文件名中提取城市名称
# file_city = file_path.split("\\")[-1].split(" ")[1]  # 提取文件名中的城市名称

# # 校验城市名称是否匹配
# if file_city.upper() != city_name.upper():
#     print(f"警告：文件名中的城市名称({file_city})与输入的城市名称({city_name})不匹配！")
#     user_input = input("是否继续处理？(y/n): ")
#     if user_input.lower() != 'y':
#         raise ValueError("用户取消处理")

In [5]:
raw_sheet = pd.read_excel(file_path, sheet_name=city_name)

In [6]:
raw_sheet.keys()

Index(['签收时间\nDelivery Time', '运单号\nTracking Number', '收派员\nDriver',
       '收派员类型\n \nDA Type', '收派员所属供应商\nDriver's Supplier',
       '签收网点\nDelivered Station\nDelivered Station',
       '商家上传重量\nVendor Uploaded Weight', '路由码\nRoute Code', '邮编\nPostcode',
       '收件城市\nCity', '计费\nCharges'],
      dtype='object')

In [7]:
supplier_key = None
signed_time_key = None
receiver_key = None
fee_key = None
for key in raw_sheet.keys():
    if "收派员所属供应商" in key:
        supplier_key = key
    if "签收时间" in key:
        signed_time_key = key
    if "收派员" in key and "driver" in key.lower() and "供应商" not in key:
        receiver_key = key
    if "计费" in key:
        fee_key = key
print(f"""寻找到的key: \n
      收派员所属供应商： {supplier_key}, \n
      签收时间： {signed_time_key}, \n
      收派员： {receiver_key}, \n
      计费： {fee_key}""")

寻找到的key: 

      收派员所属供应商： 收派员所属供应商
Driver's Supplier, 

      签收时间： 签收时间
Delivery Time, 

      收派员： 收派员
Driver, 

      计费： 计费
Charges


In [8]:
# raw_sheet = raw_sheet[raw_sheet[supplier_key]=="BIN POINT ENTREATMENT PTY LTD"]

In [9]:
raw_sheet = raw_sheet[[signed_time_key, receiver_key, fee_key]]

In [10]:
dates = raw_sheet[signed_time_key].unique()
dates = np.sort(dates)

dates = [str(date)[:10] for date in dates]
folder_name = "_".join([dates[0], dates[-1]])
project_path = os.path.join("output", city_name, folder_name)
project_cache_path = os.path.join(project_path, "cache")
os.makedirs(project_path, exist_ok=True)
os.makedirs(project_cache_path, exist_ok=True)

In [11]:
fees = raw_sheet[fee_key].unique()
fees.sort()

### 获取当周汇总数据

In [12]:
raw_sheet.shape

(37395, 3)

In [13]:
summary_sheet = raw_sheet[fee_key].value_counts().sort_index()

In [14]:
summary_sheet.name = '数量'

new_data = pd.Series([summary_sheet.sum()], index=['Total'], name='数量')  # 新行数据
summary_sheet = pd.concat([summary_sheet, new_data])  # 使用 pd.concat 添加新行
summary_sheet.index.name = '单价'

In [15]:
summary_sheet.head(20)

单价
2.2      36272
3.0        930
5.0        136
8.0         57
Total    37395
Name: 数量, dtype: int64

In [16]:
summary_sheet.to_csv(f'{project_path}/week_summary.csv')

### 获取每位派送员派送详情

In [17]:
per_person_sum = raw_sheet[receiver_key].value_counts()

In [18]:
per_person_sum.name = '总数'

In [19]:
per_person_sum = pd.DataFrame(per_person_sum)
per_person_sum = per_person_sum.reset_index()

In [20]:
import pandas as pd
import re

# 定义函数，提取公司简称、账号 ID 和账号名称
def extract_account_info(account_str):
    match = re.match(r"^([A-Z]+(?:-[A-Z]+)*)-(\w+)-(.+)$", account_str)
    if match:
        company_short = match.group(1)
        account_id = match.group(2)
        account_name = match.group(3)
        return company_short, account_id, account_name
    return None, None, account_str  # 仅有姓名时，填充 None


In [21]:
# per_person_sum['name_id'] = ["-".join(x.split("-")[:2]) for x in per_person_sum.index.tolist()]
per_person_sum[['company_short', 'account_id', 'account_name']] = per_person_sum[receiver_key].apply(lambda x: pd.Series(extract_account_info(x)))

In [22]:
len(per_person_sum.index.unique())

69

In [23]:
per_person_sum.head()

,收派员\nDriver,总数,company_short,account_id,account_name
0,BNE-BPE-5564-PATEL VED,1628,BNE-BPE,5564,PATEL VED
1,BNE-BPE-5588-ZHOU DIZHEN,1289,BNE-BPE,5588,ZHOU DIZHEN
2,BNE-BPE-5565-PATEL TAXAYKUMAR PRAVINBHAI,1251,BNE-BPE,5565,PATEL TAXAYKUMAR PRAVINBHAI
3,BNE-BPE-5546-DHARMIKKUMAR A PATEL,1212,BNE-BPE,5546,DHARMIKKUMAR A PATEL
4,BNE-BPE-7446-Man Lam,1205,BNE-BPE,7446,Man Lam


In [60]:
# 找到新增的名字
name_sheet[['company_short', 'account_id', 'account_name']] = name_sheet['account'].apply(lambda x: pd.Series(extract_account_info(x)))
per_person_sum["account_id_name"] = per_person_sum.apply(lambda row: 
    f"{row['account_id']}-{row['account_name']}".lower() if row["account_id"] else row["account_name"].lower(), axis=1)
per_person_sum['account_name'] = per_person_sum['account_name'].apply(lambda x: x.strip().lower())

name_sheet["account_id_name"] = name_sheet.apply(lambda row: 
    f"{row['account_id']}-{row['account_name']}".lower() if row["account_id"] else row["account_name"].lower(), axis=1)

name_sheet['account_name'] = name_sheet['account_name'].apply(lambda x: x.strip().lower())


dedup_key = 'account_name'

duplicate_mask = name_sheet.duplicated(subset=[dedup_key], keep=False)
if duplicate_mask.any():
    # 获取所有重复的记录
    duplicates = name_sheet[duplicate_mask].copy()
    
    # 按account_id_name分组，对每组进行处理
    for account_id_name, group in duplicates.groupby(dedup_key):
        if len(group) > 1:  # 如果有重复记录
            # 检查是否有BPE记录
            bpe_record = group[group['company_short'] == 'BPE']
            if not bpe_record.empty:
                # 如果有BPE记录，删除其他记录
                name_sheet = name_sheet[~((name_sheet[dedup_key] == account_id_name) & 
                                        (name_sheet['company_short'] != 'BPE'))]
            else:
                # 如果没有BPE记录，保留第一条记录
                name_sheet = name_sheet[~((name_sheet[dedup_key] == account_id_name) & 
                                        (name_sheet.index != group.index[0]))]


new_names_id = set(per_person_sum[dedup_key].tolist()) - set(name_sheet[dedup_key].tolist())
new_names_id = list(new_names_id)



In [62]:
# 添加到name_sheet
if new_names_id:  # only proceed if there are new names
    new_names = []
    for name_id in new_names_id:
        # Get the full name from per_person_sum DataFrame
        new_person_record = per_person_sum[per_person_sum[dedup_key]==name_id]
        company_short, account_id, account_name = extract_account_info(new_person_record[receiver_key].values[0])
        new_names.append({
            'account': new_person_record[receiver_key].values[0],
            'company_short': company_short,
            'account_id': account_id,
            'account_name': account_name
        })
    
    new_name_sheet = pd.DataFrame(new_names)
    name_sheet = pd.concat([name_sheet, new_name_sheet], ignore_index=True)

In [63]:
name_sheet['account'].drop_duplicates().to_csv(name_sheet_path, index=False)

In [64]:
name_sheet.head()

,account,company_short,account_id,account_name,account_id_name
0,BNE-BPE-5544-Dharmik Sureshbhai PATEL,BNE-BPE,5544,dharmik sureshbhai patel,5544-dharmik sureshbhai patel
1,BNE-BPE-5500-Dipak BUDHATHOKI,BNE-BPE,5500,dipak budhathoki,5500-dipak budhathoki
2,BNE-BPE-5501-BILAL MUHAMMAD,BNE-BPE,5501,bilal muhammad,5501-bilal muhammad
3,BNE-BPE-5502-Bikalpa DAWADI,BNE-BPE,5502,bikalpa dawadi,5502-bikalpa dawadi
4,BNE-BPE-5504-Chu Feng LEE,BNE-BPE,5504,chu feng lee,5504-chu feng lee


In [82]:
name_join_sheet = pd.merge(name_sheet, per_person_sum, left_on='account_name', right_on="account_name", how='left', suffixes=('_obs', ''), sort=False)
name_join_sheet.fillna(0, inplace=True)
name_join_sheet['总数'] = name_join_sheet['总数'].astype(int) # 将总数列转换为整数
# name_sheet = name_sheet.to_json(orient='records')
# name_sheet = json.loads(name_sheet)

In [83]:
name_join_sheet['总数'].sum()

37395

In [84]:
len(name_join_sheet['account_name'].unique())


91

In [85]:
name_join_sheet.head()

,account,company_short_obs,account_id_obs,account_name,account_id_name_obs,收派员\nDriver,总数,company_short,account_id,account_id_name
0,BNE-BPE-5544-Dharmik Sureshbhai PATEL,BNE-BPE,5544,dharmik sureshbhai patel,5544-dharmik sureshbhai patel,BNE-BPE-5544-Dharmik Sureshbhai PATEL,370,BNE-BPE,5544,5544-dharmik sureshbhai patel
1,BNE-BPE-5500-Dipak BUDHATHOKI,BNE-BPE,5500,dipak budhathoki,5500-dipak budhathoki,BNE-BPE-5500-Dipak BUDHATHOKI,733,BNE-BPE,5500,5500-dipak budhathoki
2,BNE-BPE-5501-BILAL MUHAMMAD,BNE-BPE,5501,bilal muhammad,5501-bilal muhammad,0,0,0,0,0
3,BNE-BPE-5502-Bikalpa DAWADI,BNE-BPE,5502,bikalpa dawadi,5502-bikalpa dawadi,BNE-BPE-5502-Bikalpa DAWADI,455,BNE-BPE,5502,5502-bikalpa dawadi
4,BNE-BPE-5504-Chu Feng LEE,BNE-BPE,5504,chu feng lee,5504-chu feng lee,BNE-BPE-5504-Chu Feng LEE,911,BNE-BPE,5504,5504-chu feng lee


In [86]:
per_person_sum['总数'].sum()

37395

In [87]:
if name_join_sheet['总数'].sum() != summary_sheet.sum()//2:
    print(f"总数不匹配: {name_join_sheet['总数'].sum()} != {summary_sheet.sum()//2}")
else:
    print(f"总数匹配: {name_join_sheet['总数'].sum()} == {summary_sheet.sum()//2}")


总数匹配: 37395 == 37395


In [88]:
# import tkinter as tk
# from tkinter import messagebox
# import win32gui
# import win32con

# # 数据校验
# total_deliveries = name_sheet['总数'].sum()
# total_summary = summary_sheet.sum()//2

# if total_deliveries != total_summary:
#     # 创建弹窗
#     root = tk.Tk()
#     # root.withdraw()  # 隐藏主窗口
    
#     # 显示错误消息
#     error_dialog = messagebox.showerror(
#         "数据校验错误",
#         f"数据不匹配！\n\n"
#         f"派送员总数: {total_deliveries}\n"
#         f"汇总表总数: {total_summary}\n"
#         f"差异数量: {abs(total_deliveries - total_summary)}\n\n"
#         f"请检查以下可能的原因：\n"
#         f"1. 是否有派送员数据未正确统计\n"
#         f"2. 是否有重复计算的情况\n"
#         f"3. 是否有数据丢失"
#     )
    
#     # 获取错误对话框的句柄并设置为最顶层
#     hwnd = win32gui.GetForegroundWindow()
#     win32gui.SetWindowPos(hwnd, win32con.HWND_TOPMOST, 0, 0, 0, 0, 
#                          win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)
    
#     # 询问是否继续
#     confirm_dialog = messagebox.askyesno("确认", "是否继续处理？")
    
#     # # 获取确认对话框的句柄并设置为最顶层
#     # hwnd = win32gui.GetForegroundWindow()
#     # win32gui.SetWindowPos(hwnd, win32con.HWND_TOPMOST, 0, 0, 0, 0, 
#     #                      win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)
    
#     if not confirm_dialog:
#         raise ValueError("用户取消处理")
    
#     root.destroy()  # 关闭窗口
# else:
#     print(f"✅ 数据校验通过！总数匹配: {total_deliveries} == {total_summary}")

In [89]:
# print(counts.to_json(force_ascii=False))
# signed_time_key = None
# receiver_key = None
# fee_key = None
raw_sheet[signed_time_key] = raw_sheet[signed_time_key].apply(lambda x: str(x)[:10])
counts = raw_sheet.groupby([receiver_key, signed_time_key, fee_key]).size().reset_index(name='记录数')

In [90]:
# 转换为嵌套 JSON
nested_json = (
    counts.reset_index()  # 将 MultiIndex 转为普通列
    .groupby(receiver_key)  # 按收派员名字分组
    .apply(lambda x: x.groupby(signed_time_key)  # 在组内按签收时间分组
           .apply(lambda y: y.set_index(fee_key)['记录数'].to_dict())  # 将计费和对应数量转为字典
           .to_dict())  # 将签收时间分组转为字典
    .to_dict()  # 将收派员编号分组转为字典
)

In [91]:
name_join_sheet['data'] = name_join_sheet['account'].apply(lambda x: nested_json.get(x, {}))

In [92]:
# columns = ["序号", "编号", "ACCOUNT"]
columns = ["序号", "ACCOUNT"]
sub_columns = [""] * len(columns)
# 定义表头
for d in dates:
    columns.append(d)
    columns.extend([""] * (len(fees) - 1))

    sub_columns.extend(fees)

columns.append("Total")
sub_columns.append("")
# 创建多级表头
header = pd.MultiIndex.from_tuples(list(zip(columns, sub_columns)))

In [93]:
name_join_sheet.head()

,account,company_short_obs,account_id_obs,account_name,account_id_name_obs,收派员\nDriver,总数,company_short,account_id,account_id_name,data
0,BNE-BPE-5544-Dharmik Sureshbhai PATEL,BNE-BPE,5544,dharmik sureshbhai patel,5544-dharmik sureshbhai patel,BNE-BPE-5544-Dharmik Sureshbhai PATEL,370,BNE-BPE,5544,5544-dharmik sureshbhai patel,"{'2025-04-01': {2.2: 80}, '2025-04-05': {2.2: ..."
1,BNE-BPE-5500-Dipak BUDHATHOKI,BNE-BPE,5500,dipak budhathoki,5500-dipak budhathoki,BNE-BPE-5500-Dipak BUDHATHOKI,733,BNE-BPE,5500,5500-dipak budhathoki,"{'2025-03-31': {2.2: 28}, '2025-04-01': {2.2: ..."
2,BNE-BPE-5501-BILAL MUHAMMAD,BNE-BPE,5501,bilal muhammad,5501-bilal muhammad,0,0,0,0,0,{}
3,BNE-BPE-5502-Bikalpa DAWADI,BNE-BPE,5502,bikalpa dawadi,5502-bikalpa dawadi,BNE-BPE-5502-Bikalpa DAWADI,455,BNE-BPE,5502,5502-bikalpa dawadi,"{'2025-04-03': {2.2: 147}, '2025-04-04': {2.2:..."
4,BNE-BPE-5504-Chu Feng LEE,BNE-BPE,5504,chu feng lee,5504-chu feng lee,BNE-BPE-5504-Chu Feng LEE,911,BNE-BPE,5504,5504-chu feng lee,"{'2025-03-31': {2.2: 130}, '2025-04-01': {2.2:..."


In [94]:
full_data = []

for idx, person in name_join_sheet.iterrows():
    data = [idx+1, person['account']]
    for d in dates:
        if d in person['data']:
            for f in fees:
                if f in person['data'][d]:
                    data.append(person['data'][d].get(f, ""))
                else:
                    data.append("")
        else:
            data.extend([""] * len(fees))
    data.append(person['总数'])
    full_data.append(data)

In [95]:
# 添加汇总行
total_row = ["Total", "总计"]
total_by_date_fee = {}  # 用于存储每个日期和费用类型的总和

# 初始化汇总数据结构
for d in dates:
    total_by_date_fee[d] = {f: 0 for f in fees}

# 计算每个日期和费用类型的总和
for person in name_join_sheet.itertuples():
    for d in dates:
        if hasattr(person, 'data') and d in person.data:
            for f in fees:
                if f in person.data[d]:
                    total_by_date_fee[d][f] += person.data[d][f]

# 构建汇总行数据
for d in dates:
    for f in fees:
        total_row.append(total_by_date_fee[d][f])

# 添加总计数量
total_row.append(name_join_sheet['总数'].sum())

# 将汇总行添加到full_data
full_data.append(total_row)

In [96]:
# 创建 DataFrame
df = pd.DataFrame(full_data, columns=header)

In [97]:
from openpyxl import load_workbook

df.to_excel(f"{project_cache_path}/output.xlsx", index=True)
# 加载并移除空行
wb = load_workbook(f"{project_cache_path}/output.xlsx")
ws = wb.active

# 删除空行（通常是第3行）
ws.delete_rows(3)
# ws.delete_cols(0)

# 保存调整后的文件
wb.save(f"{project_path}/per_person_details.xlsx")
print(f"文件已保存至 {project_path}/per_person_details.xlsx")
wb.close()

文件已保存至 output\BNE\2025-03-31_2025-04-06/per_person_details.xlsx
